In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from PIL import Image
from keras import utils
%matplotlib inline

In [ ]:
train_df = pd.read_csv('../data/train.csv')

In [ ]:
#get targets to write out
y_train = train_df['sirna'].values
y_train = utils.to_categorical(y_train)
print(y_train.shape)

In [ ]:
def parse_all_train():
    dataset = 'train'
    experiments = train_df['experiment'].values
    plates = train_df['plate'].values
    wells = train_df['well'].values
    sites = np.array([1,2])
    channels=np.array([1,2,3,4,5,6])
    
    start = 0
    end = 36000
    
    for site in sites:
        for k in range(start, end, 1000):
            print("Current cycle: ", k)
            site_img = np.empty((1000, 512, 512, 6), dtype=np.float16)
            j=0
            for i  in tqdm_notebook(range(k, k+site_img.shape[0])):
                for channel in channels:
                    path = f'../data/{dataset}/{experiments[i]}/Plate{plates[i]}/{wells[i]}_s{site}_w{channel}.png'
                    print(f'current image: {j}')
                    site_img[j, :, :, channel-1] = Image.open(path)
                j+=1
                    
            #normalize here
            site_img /= 255

            print("x_train shape: ", site_img.shape)

            print("Writing to file...")
            np.save(f'../data/train_parsed/x_train_s{site}_{k}', site_img, allow_pickle=True)
            np.save(f'../data/train_parsed/y_train_s{site}_{k}', y_train[k:k+site_img.shape[0]], allow_pickle=True)

In [ ]:
parse_all_train()